In [ ]:
# Model Fusion RGB and GPS Transformers: DeepSense Scenario 36 64 Beams!

# Average Top-1 accuracy 0.3060483870967742
# Average Top-3 accuracy 0.48104838709677417
# Average Top-5 accuracy 0.535483870967742
# Average Top-7 accuracy 0.5762096774193548
# Average Top-9 accuracy 0.6133064516129032
# Average Top-11 accuracy 0.6475806451612903
# Average Top-13 accuracy 0.6762096774193549
# Average Top-15 accuracy 0.7044354838709678

In [ ]:
# !pip install timm

In [1]:
import os
import random
import datetime
import sys
import shutil

import torch
import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transf
from torchvision import transforms, utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from skimage import io
import ast

In [2]:
# Training Hyper-parameters
batch_size = 16 # 32 out of memory problem!
val_batch_size = 1
lr = 0.001
decay = 1e-4
num_epochs = 20 # After 10 epoch, the accuracy remains same, but running 20 to get more loss!
train_size = [1]

# Hyperparameters for our network
input_size = 2
node = 512
output_size = 65

In [3]:
#image transform
img_resize = transf.Resize((224, 224))
img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
image_proc_pipe = transf.Compose(
    [transf.ToPILImage(),
     img_resize,
     transf.ToTensor(),
     img_norm]
)


#POS transform
pos_proc_pipe = transf.Compose(
    [
        transf.ToTensor()
    ]
)

In [4]:
# year month day
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

07-26-2024
23_53


In [5]:
pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime
print(pwd)
# Determine whether the folder already exists
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario36_64-Beams\Main_Folder//saved_folder//07-26-2024_23_53


In [6]:
save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory)

isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

In [7]:
# !pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  2% | 11% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  2% | 12% |


In [8]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0))

CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce RTX 2080 Super with Max-Q Design


In [10]:
torch.cuda.memory_allocated()

0

In [11]:
torch.cuda.max_memory_allocated()

0

In [12]:
import torch
import torch.nn as nn
import torch.optim as optimizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transf
import timm
import pandas as pd
import numpy as np
import ast
from skimage import io

class FusionModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(FusionModel, self).__init__()
        self.nclass = num_classes
        self.nf = num_features

        # Using MaxViT from timm
        self.maxvit = timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True)
        self.maxvit_fc = nn.Linear(self.maxvit.num_features, 128)

        # Using a simple MLP for the beam prediction
        self.positional_fc = nn.Sequential(
            nn.Linear(self.nf, 128),
            nn.ReLU(),
            nn.Linear(128, 128)
        )

        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, self.nclass)

    def forward(self, inputs):
        # Process image data through MaxViT
        x = self.maxvit.forward_features(inputs[0])
        x = x.mean(dim=[2, 3])  # Global average pooling
        x = self.maxvit_fc(x)

        # Process positional data through the simple MLP
        y = self.positional_fc(inputs[1].reshape([-1, self.nf]))

        # Concatenate and pass through fully connected layers
        x = torch.cat([x, y], dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

### Data Preparation

def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    for idx, row in f.iterrows():
        data = list(row.values[1:])
        data_samples.append(data)
    return data_samples

class DataFeed(Dataset):
    def __init__(self, image_dir, pos_dir, nat_sort=False, image_transform=None, pos_transform=None, init_shuflle=True):
        self.root1 = image_dir
        self.root2 = pos_dir
        self.samples1 = create_samples(self.root1, shuffle=init_shuflle, nat_sort=nat_sort)
        self.samples2 = create_samples(self.root2, shuffle=init_shuflle, nat_sort=nat_sort)
        self.transform1 = image_transform
        self.transform2 = pos_transform

    def __len__(self):
        return len(self.samples1)

    def __getitem__(self, idx):
        sample1 = self.samples1[idx]
        img = io.imread(sample1[0])
        if self.transform1:
            img = self.transform1(img)
        label = sample1[1]

        sample2 = self.samples2[idx]
        pos_val = sample2[:1]
        pos_val = ast.literal_eval(pos_val[0])
        pos_val = np.asarray(pos_val)

        return ([img, pos_val], label)

### Image Transform
img_resize = transf.Resize((224, 224))
img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
image_proc_pipe = transf.Compose(
    [transf.ToPILImage(),
     img_resize,
     transf.ToTensor(),
     img_norm]
)

### POS Transform
pos_proc_pipe = transf.Compose(
    [
        transf.ToTensor()
    ]
)

image_train_dir = './scenario36_64_img_beam_train.csv'
image_val_dir = './scenario36_64_img_beam_val.csv'
pos_train_dir = './scenario36_64_pos_beam_train.csv'
pos_val_dir = './scenario36_64_pos_beam_val.csv'

train_loader = DataLoader(DataFeed(image_train_dir, pos_train_dir, image_transform=image_proc_pipe),
                          batch_size=batch_size,
                          shuffle=False)
val_loader = DataLoader(DataFeed(image_val_dir, pos_val_dir, image_transform=image_proc_pipe),
                        batch_size=val_batch_size,
                        shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = FusionModel(num_features=input_size, num_classes=output_size).to(device)

### Training and Validation Logic
val_acc = []

criterion = nn.CrossEntropyLoss()
opt = optimizer.Adam(model.parameters(), lr=lr, weight_decay=decay)
LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15, 25, 40], gamma=0.1, last_epoch=-1)

top_1 = np.zeros((1, len(train_size)))
top_3 = np.zeros((1, len(train_size)))
top_5 = np.zeros((1, len(train_size)))
top_7 = np.zeros((1, len(train_size)))
top_9 = np.zeros((1, len(train_size)))
top_11 = np.zeros((1, len(train_size)))
top_13 = np.zeros((1, len(train_size)))
top_15 = np.zeros((1, len(train_size)))
acc_loss = 0
itr = []

for idx, n in enumerate(train_size):
    print('Training size is {}'.format(n))
    net = model
    criterion = nn.CrossEntropyLoss()
    opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
    LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15, 25, 40], gamma=0.1, last_epoch=-1)

    count = 0
    running_loss = []
    running_top1_acc = []
    running_top3_acc = []
    running_top5_acc = []
    running_top7_acc = []
    running_top9_acc = []
    running_top11_acc = []
    running_top13_acc = []
    running_top15_acc = []
    best_accuracy = 0

    for epoch in range(num_epochs):
        print('Epoch No. ' + str(epoch + 1))
        skipped_batches = 0
        epoch_train_loss = 0  # To track the training loss for the epoch # Added
        net.train()
        for tr_count, (pos_data, beam_val) in enumerate(train_loader):
            data = [pos_data[0].to(device), pos_data[1].type(torch.Tensor).to(device)]
            label = beam_val.type(torch.LongTensor).to(device)
            opt.zero_grad()
            out = net(data)
            loss = criterion(out, label)
            loss.backward()
            opt.step()
            batch_loss = loss.item()
            acc_loss += batch_loss
            epoch_train_loss += batch_loss  # Accumulate batch loss for the epoch # Added
            count += 1
            if count % 100 == 0:
                print('Training-Batch No.' + str(count))
                running_loss.append(batch_loss)
                itr.append(count)
                print('Loss = ' + str(running_loss[-1]))

        epoch_train_loss /= len(train_loader)  # Calculate average training loss for the epoch # Added
        print(f'Epoch {epoch + 1} Training Loss: {epoch_train_loss:.4f}') # Added
        
        print('Start Training and Validation')
        ave_top1_acc = 0
        ave_top3_acc = 0
        ave_top5_acc = 0
        ave_top7_acc = 0
        ave_top9_acc = 0
        ave_top11_acc = 0
        ave_top13_acc = 0
        ave_top15_acc = 0
        
        val_loss = 0  # To track the validation loss # Added

        ind_ten = torch.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device=device)

        top1_pred_out = []
        top3_pred_out = []
        top5_pred_out = []
        top7_pred_out = []
        top9_pred_out = []
        top11_pred_out = []
        top13_pred_out = []
        top15_pred_out = []

        total_count = 0
        gt_beam = []

        # To track validation loss
        val_loss = 0

        net.eval()
        with torch.no_grad():
            for val_count, (pos_data, beam_val) in enumerate(val_loader):
                data = [pos_data[0].to(device), pos_data[1].type(torch.Tensor).to(device)]
                labels = beam_val.type(torch.LongTensor).to(device)
                gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
                total_count += labels.size(0)
                out = net(data)
                loss = criterion(out, labels)  # Calculate validation loss
                val_loss += loss.item()  # Accumulate validation loss
                
                val_batch_loss = criterion(out, labels).item()  # Calculate validation loss for the batch # Added
                val_loss += val_batch_loss  # Accumulate batch loss for the epoch # Added
                
                _, top_1_pred = torch.max(out, dim=1)
                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
                sorted_out = torch.argsort(out, dim=1, descending=True)

                top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

                top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

                top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

                top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())


                top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

                top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

                top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)

                batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
                batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
                batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
                batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
                batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
                batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
                batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
                batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()
                
        val_loss /= len(val_loader)  # Calculate average validation loss for the epoch # Added
        print(f'Epoch {epoch + 1} Validation Loss: {val_loss:.4f}') # Added

        print("Total training examples:", total_count)
        running_top1_acc.append(ave_top1_acc / total_count)
        running_top3_acc.append(ave_top3_acc / total_count)
        running_top5_acc.append(ave_top5_acc / total_count)
        running_top7_acc.append(ave_top7_acc / total_count)
        running_top9_acc.append(ave_top9_acc / total_count)
        running_top11_acc.append(ave_top11_acc / total_count)
        running_top13_acc.append(ave_top13_acc / total_count)
        running_top15_acc.append(ave_top15_acc / total_count)

        # print('Validation Loss: {}'.format(val_loss / len(val_loader)))  # Print the validation loss
        print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
        print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
        print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
        print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
        print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
        print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
        print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
        print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

        cur_accuracy = running_top1_acc[-1]

        print("Current accuracy:", cur_accuracy)
        print("Best accuracy:", best_accuracy)
        if cur_accuracy > best_accuracy:
            print("Saving the best model")
            net_name = checkpoint_directory + '/fused_transfomer_beam_pred'
            torch.save(net.state_dict(), net_name)
            best_accuracy = cur_accuracy
        print("Updated best accuracy:", best_accuracy)

        print("Saving the predicted values in a CSV file")
        file_to_save = f'{save_directory}/topk_pred_beam_val_after_{epoch+1}th_epoch.csv'
        indx = np.arange(1, len(top1_pred_out) + 1, 1)
        df1 = pd.DataFrame()
        df1['index'] = indx
        df1['link_status'] = gt_beam
        df1['top1_pred'] = top1_pred_out
        df1['top3_pred'] = top3_pred_out
        df1['top5_pred'] = top5_pred_out
        df1['top7_pred'] = top7_pred_out
        df1['top9_pred'] = top9_pred_out
        df1['top11_pred'] = top11_pred_out
        df1['top13_pred'] = top13_pred_out
        df1['top15_pred'] = top15_pred_out
        df1.to_csv(file_to_save, index=False)

        LR_sch.step()

    top_1[0, idx] = running_top1_acc[-1]
    top_3[0, idx] = running_top3_acc[-1]
    top_5[0, idx] = running_top5_acc[-1]
    top_7[0, idx] = running_top7_acc[-1]
    top_9[0, idx] = running_top9_acc[-1]
    top_11[0, idx] = running_top11_acc[-1]
    top_13[0, idx] = running_top13_acc[-1]
    top_15[0, idx] = running_top15_acc[-1]

Training size is 1
Epoch No. 1
Training-Batch No.100
Loss = 3.6877543926239014
Training-Batch No.200
Loss = 3.3775858879089355
Training-Batch No.300
Loss = 2.4910552501678467
Training-Batch No.400
Loss = 3.5379505157470703
Training-Batch No.500
Loss = 2.074571371078491
Training-Batch No.600
Loss = 3.0097644329071045
Training-Batch No.700
Loss = 2.888113498687744
Training-Batch No.800
Loss = 2.5640764236450195
Training-Batch No.900
Loss = 2.6636219024658203
Start Validation
Total validation examples: 7440
Validation Loss: 3.137246430866947
Average Top-1 accuracy 0.22836021505376344
Average Top-3 accuracy 0.37553763440860216
Average Top-5 accuracy 0.4561827956989247
Average Top-7 accuracy 0.49838709677419357
Average Top-9 accuracy 0.5313172043010753
Average Top-11 accuracy 0.5625
Average Top-13 accuracy 0.5897849462365592
Average Top-15 accuracy 0.6173387096774193
Current accuracy: 0.22836021505376344
Best accuracy: 0
Saving the best model
Updated best accuracy: 0.22836021505376344
Savin

In [13]:
print('Start Validation')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(val_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = np.arange(1, len(top1_pred_out)+1, 1)
df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Validation
total examples are 7440
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.27876344086021504
Average Top-3 accuracy 0.4399193548387097
Average Top-5 accuracy 0.4978494623655914
Average Top-7 accuracy 0.5365591397849462
Average Top-9 accuracy 0.5678763440860215
Average Top-11 accuracy 0.5973118279569892
Average Top-13 accuracy 0.6237903225806452
Average Top-15 accuracy 0.6454301075268817
Saving the predicted value in a csv file


In [ ]:
# Testing

In [14]:
# Load the model checkpoint
image_test_dir = './scenario36_64_img_beam_test.csv'
pos_test_dir = './scenario36_64_pos_beam_test.csv'

# Load the test data
test_data = pd.read_csv(image_test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

checkpoint_path = f'{checkpoint_directory}/fused_transfomer_beam_pred'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
net = model.cuda()

In [15]:
test_loader = DataLoader(DataFeed(image_test_dir, pos_test_dir, image_transform=image_proc_pipe),
                            batch_size=val_batch_size,
                            #num_workers=8,
                            shuffle=False)

In [16]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(test_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total test examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval_Test.csv'

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
# link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
# org = test_data['original_index'].tolist()
# pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data  # Add the link_status column
df2['original_unit1_pwr3'] = pwr_60ghz # Add the original_unit1_pwr_60ghz column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out

df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Testing
total test examples are 2480
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.3060483870967742
Average Top-3 accuracy 0.48104838709677417
Average Top-5 accuracy 0.535483870967742
Average Top-7 accuracy 0.5762096774193548
Average Top-9 accuracy 0.6133064516129032
Average Top-11 accuracy 0.6475806451612903
Average Top-13 accuracy 0.6762096774193549
Average Top-15 accuracy 0.7044354838709678
Saving the predicted value in a csv file
